# Bagging test

In [10]:
import pandas as pd
import numpy as np
from random import sample
from scipy.stats import binom
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sksurv.tree import SurvivalTree

Load data:

In [13]:
whitehall = pd.read_csv('../Practicals/whitehall.csv', index_col = 0, squeeze = True)

In [15]:
whitehall

,all,chd,sbp,chol,grade4,smok,agein,grade,cholgrp,sbpgrp,timein,timeout,timebth
id,,,,,,,,,,,,,
5001,0,0,120,273,3,4,47.48255,2,4,2,13sep1967,30jan1987,20mar1920
5019,0,0,118,234,1,3,56.10678,1,3,1,13sep1967,30jan1987,06aug1911
5038,0,0,147,295,2,3,51.44422,1,4,3,13sep1967,30jan1987,03apr1916
5039,0,0,92,210,2,1,51.11567,1,3,1,13sep1967,30jan1987,02aug1916
5042,1,0,128,287,2,2,55.48528,1,4,2,13sep1967,12apr1983,20mar1912
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24143,0,0,139,150,4,4,53.27036,2,2,2,09jan1970,30jan1987,02oct1916
24163,1,0,137,205,4,4,62.27789,2,3,2,09jan1970,16sep1984,29sep1907
24167,1,1,174,185,3,2,59.12389,2,2,4,09jan1970,01mar1980,25nov1910


In [46]:
# Size of dataset
n = len(y)
n

1677

## Subcohort sample
We create a simulated subcohort sample in this section.

In [25]:
# Sample of population we are using for the subcohort:
samp_prop = 0.2

In [72]:
#Subcohort:
whitehall_subcohort = whitehall.sample(frac = samp_prop)
whitehall_subcohort

,all,chd,sbp,chol,grade4,smok,agein,grade,cholgrp,sbpgrp,timein,timeout,timebth
id,,,,,,,,,,,,,
22120,0,0,143,150,2,2,47.86037,1,2,3,30jun1969,30jan1987,20aug1921
10201,0,0,115,201,2,4,55.37029,1,3,1,15may1968,30jan1987,01jan1913
12106,0,0,121,238,1,2,58.65572,1,3,2,19jul1968,30jan1987,21nov1909
22029,0,0,138,230,2,5,44.30390,1,3,2,27jun1969,30jan1987,07mar1925
5337,1,1,115,216,2,4,40.19439,1,3,1,20sep1967,13jan1987,12jul1927
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16240,0,0,178,195,2,2,60.71184,1,2,4,27nov1968,30jan1987,13mar1908
19697,0,0,166,285,2,1,57.03217,1,4,4,31mar1969,30jan1987,18mar1912
17318,0,0,131,185,2,2,43.35661,1,2,2,15jan1969,30jan1987,07sep1925


We also need the case dataset:

In [74]:
cases = whitehall.query('chd == 1')
cases

,all,chd,sbp,chol,grade4,smok,agein,grade,cholgrp,sbpgrp,timein,timeout,timebth
id,,,,,,,,,,,,,
5078,1,1,144,268,2,2,47.72348,1,4,3,15sep1967,29nov1983,25dec1919
5337,1,1,115,216,2,4,40.19439,1,3,1,20sep1967,13jan1987,12jul1927
5346,1,1,146,190,2,2,56.87337,1,2,3,20sep1967,12feb1981,06nov1910
5350,1,1,118,202,3,3,54.31348,2,3,1,20sep1967,12jul1980,28may1913
5461,1,1,127,256,1,1,55.38398,1,4,2,25sep1967,26may1983,08may1912
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23261,1,1,158,215,3,5,67.84394,2,3,3,15sep1969,09aug1971,12nov1901
23489,1,1,155,150,2,3,61.44011,1,2,3,03dec1969,14jun1976,24jun1908
23610,1,1,132,138,3,4,57.04312,2,1,2,08dec1969,21may1979,22nov1912


## Bootstrap sampling function
We want to create a function that samples from the subcohort in a way that mimics sampling from the population, but while utilising all the casedata we have.

Proportion of cohort that are cases:

In [79]:
prop_cases = len(whitehall.query('chd == 1'))/n
prop_cases

0.09183064997018485

In [98]:
# Function samples cases and non cases in desired fashion
def sampler(subcohort, cases, prop_cases, boot_prop):
    m = len(subcohort)
    n_cases = binom.rvs(round(boot_prop*m),prop_cases)
    
    samp = pd.concat([subcohort.query('chd == 0').sample(round(boot_prop*m) - n_cases),cases.sample(n_cases)])
    
    return(samp)

## Bagging
Now to combine all these functions together into a bootstrap classifier.

In [195]:
test_sample = sampler(whitehall_subcohort, cases, prop_cases, 0.5)
test_sample

,all,chd,sbp,chol,grade4,smok,agein,grade,cholgrp,sbpgrp,timein,timeout,timebth
id,,,,,,,,,,,,,
19371,0,0,146,165,1,2,52.67899,1,2,3,21mar1969,30jan1987,15jul1916
17899,0,0,152,190,2,5,56.65982,1,2,3,12feb1969,30jan1987,16jun1912
15227,0,0,126,135,2,5,48.04107,1,1,2,04nov1968,30jan1987,20oct1920
17390,0,0,152,190,2,5,47.34291,1,2,3,17jan1969,30jan1987,14sep1921
20717,0,0,137,185,4,3,47.52361,2,2,2,30apr1969,30jan1987,20oct1921
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20908,1,1,158,145,2,2,54.70226,1,1,3,05may1969,11mar1982,21aug1914
14125,1,1,156,280,2,2,63.88501,1,4,3,02oct1968,27nov1968,13nov1904
7772,1,1,119,236,2,2,56.33949,1,3,1,31jan1968,26mar1969,29sep1911


In [196]:
clf = tree.DecisionTreeClassifier()

In [197]:
clf = clf.fit(test_sample.iloc[:,3:7],test_sample['chd'])

In [198]:
test = whitehall.sample().iloc[:,3:7]
test

,chol,grade4,smok,agein
id,,,,
18624,110,2,1,47.52088


In [199]:
y_pred = clf.predict(whitehall.iloc[:,3:7])

In [200]:
y_true = whitehall['chd']

In [201]:
accuracy_score(y_true, y_pred)

0.862850327966607

What's the accuracy when we just use the subcohort?

In [192]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(whitehall_subcohort.iloc[:,3:7],whitehall_subcohort['chd'])

In [193]:
y_pred = clf.predict(whitehall.iloc[:,3:7])

In [2]:
accuracy_score(y_true,y_pred)

NameError: name 'y_true' is not defined

In [13]:
print(inspect.getsource(SurvivalTree))

class SurvivalTree(BaseEstimator, SurvivalAnalysisMixin):
    """A survival tree.

    The quality of a split is measured by the
    log-rank splitting rule.

    See [1]_, [2]_ and [3]_ for further description.

    Parameters
    ----------
    splitter : string, optional, default: "best"
        The strategy used to choose the split at each node. Supported
        strategies are "best" to choose the best split and "random" to choose
        the best random split.

    max_depth : int or None, optional, default: None
        The maximum depth of the tree. If None, then nodes are expanded until
        all leaves are pure or until all leaves contain less than
        min_samples_split samples.

    min_samples_split : int, float, optional, default: 6
        The minimum number of samples required to split an internal node:

        - If int, then consider `min_samples_split` as the minimum number.
        - If float, then `min_samples_split` is a fraction and
          `ceil(min_sampl

In [7]:
import inspect

In [8]:
DecisionTreeClassifier()

NameError: name 'DecisionTreeClassifier' is not defined